# Crystal field and covalency effects in uranates 

It has been demostrated recently that high-energy resolution fluorescence-detection X-ray absorption spectroscopy (HERFD-XAS) technique can be used to directly probe the crystal-field splitting in the 5f shell of actinides. In addition, the HERFD-XAS technique allows us to resolve the charge-transfer satellites in actinide spectra that were hidden before, thus enhancing the sensitivity to the covalent character of the chemical bonding.

<figure>
  <img src="assets/uranates_covalency.png" alt="Uranates covalency" style="width:35%">
  <figcaption style="text-align: center; font-style: italic">1 S.M. Butorin et al., Chemistry - A European Journal 22, 9693 (2016).
</figcaption>
</figure>

In this notebook, you will calculated the absorption spectra of an U(V) ion in octahedral coordination using various levels of approximation in the Hamiltonian.

<figure>
  <img src="assets/uranates_hamiltonian.png" alt="Hamiltonian in uranates" style="width:45%">
</figure>

## Import the required packages and setup logging

In [ ]:
%matplotlib notebook

import os
import sys
import logging

import numpy as np
import matplotlib.pyplot as plt

from crispy.notebook import Calculation


logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logging.getLogger("crispy").setLevel(logging.INFO)

## Setup the calculation

In [ ]:
calc = Calculation("U5+", "Oh", "XAS", "M4,5 (3d)")

calc.xaxis.set_parameter("Lorentzian", 0.35)
calc.xaxis.set_parameter("Gaussian", 0.8)
calc.xaxis.set_parameter("Stop", 3750)
calc.xaxis.set_parameter("Start", 3725)

calc.hamiltonian.set_parameter("Fk", 0.7)
calc.hamiltonian.set_parameter("Gk", 0.7)

## Create an empty dictionary to save the calculated spectra

In [ ]:
results = {}

## Calculate the atomic spectrum

In [ ]:
calc.hamiltonian.terms

In [ ]:
calc.hamiltonian.terms.disable("Crystal Field")

In [ ]:
calc.hamiltonian.terms

In [ ]:
calc.run()

In [ ]:
data = calc.spectra.get_calculated_data()
x = data["Isotropic Absorption"]["x"] 
signal = data["Isotropic Absorption"]["signal"]
results["Atomic"] = (x, signal)

## Calculate the crystal field spectrum

The crystal field splitting can be expressed using the Wybourne’s parameters, $B_{40}$ and $B_{60}$. The energy of the f-orbitals is then equal to:

$E_{a2u} = -\frac{4}{11} B_{40} + \frac{80}{143} B_{60}$

$E_{t1u} = \frac{2}{11} B_{40} + \frac{100}{429} B_{60}$

$E_{t2u} = -\frac{2}{33} B_{40} - \frac{60}{143} B_{60}$

In [ ]:
B40 = 2.03
B60 = 0.17

Ea2u = -4 / 11 * B40 +  80 / 143 * B60
Et1u =  2 / 11 * B40 + 100 / 429 * B60
Et2u = -2 / 33 * B40 -  60 / 143 * B60

calc.hamiltonian.set_parameter("Ea2u(5f)", Ea2u)
calc.hamiltonian.set_parameter("Et1u(5f)", Et1u)
calc.hamiltonian.set_parameter("Et2u(5f)", Et2u)

calc.hamiltonian.terms.enable("Crystal Field")

In [ ]:
calc.run()

In [ ]:
data = calc.spectra.get_calculated_data()
x = data["Isotropic Absorption"]["x"] 
signal = data["Isotropic Absorption"]["signal"]
results["Crystal field"] = (x, signal)

## Calculate the ligand field spectrum

In [ ]:
calc.hamiltonian.set_parameter("Δ(5f,L1)", 4.0)
calc.hamiltonian.set_parameter("U(5f,5f)", 3.5)
calc.hamiltonian.set_parameter("U(3d,5f)", 6.0)

calc.hamiltonian.set_parameter("Va2u(5f,L1)", 1.2, hamiltonian_name="Initial Hamiltonian")
calc.hamiltonian.set_parameter("Vt1u(5f,L1)", 1.2, hamiltonian_name="Initial Hamiltonian")
calc.hamiltonian.set_parameter("Vt2u(5f,L1)", 1.2, hamiltonian_name="Initial Hamiltonian")

calc.hamiltonian.set_parameter("Va2u(5f,L1)", 0.9, hamiltonian_name="Final Hamiltonian")
calc.hamiltonian.set_parameter("Vt1u(5f,L1)", 0.9, hamiltonian_name="Final Hamiltonian")
calc.hamiltonian.set_parameter("Vt2u(5f,L1)", 0.9, hamiltonian_name="Final Hamiltonian")

calc.hamiltonian.terms.enable("5f-Ligands Hybridization (LMCT)")

calc.hamiltonian

In [ ]:
calc.run()

In [ ]:
data = calc.spectra.get_calculated_data()
x = data["Isotropic Absorption"]["x"] 
signal = data["Isotropic Absorption"]["signal"]
results["Ligand field"] = (x, signal)

## Plot the calculated spectra

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4.3))

for name, (x, signal) in results.items():
    ax.plot(x, signal, label=name)

ax.legend()
ax.grid()
ax.set_xlabel("Absorption Energy (eV)")
ax.set_ylabel("Intensity (a.u.)")
plt.tight_layout()

In [ ]:
print(calc.get_output())